<a href="https://colab.research.google.com/github/xiaochengJF/DeepLearning/blob/master/%E6%84%9F%E7%9F%A5%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
cd "/content/gdrive/My Drive/My/"

/content/gdrive/My Drive/My


In [0]:
from __future__ import print_function
from functools import reduce

In [0]:
class Perceptron(object):
    def __init__(self, input_num, activator):
        """
        初始化感知器，设置参数个数，及激活函数
        激活函数的类型为double -> double
        """
        self.activator = activator
        # 初始化权重和偏置项为0
        self.weights = [0.0] * imput_num
        self.bias = 0.0
        
    def __str__(self):
        """
        打印学习到的权重、偏置
        """
        return "weight\t:%s\nbias\t"%f\n" % (self.weights, self.bias)
    
    def predict(self, input_vec):
        """
        输入向量，输出结果
        """
        